In [22]:
# ==========================================
# Cell 1: 生成全量数据 -> 模拟 Cell 3 排序 -> 锁定顺序
# ==========================================
import pandas as pd
import numpy as np
import json
from pathlib import Path
from typing import Dict, List, Any

# --- 路径配置 ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
OUTPUT_CSV = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
OUTPUT_JSON = DATA_DIR / "fixed_ppt_order.json"

# --- 1. 特征计算 (来自 Cell 12) ---
def calculate_cluster_metrics(df_sub: pd.DataFrame, is_intensity: bool) -> Dict[int, Dict[str, Any]]:
    th_high = 6.0 if is_intensity else 0.6
    th_med = 3.0 if is_intensity else 0.3
    th_slope = 1.5 if is_intensity else 0.15

    df_calc = df_sub[(df_sub['年份'] >= 2005) & (df_sub['年份'] <= 2023)]
    # 注意：groupby 默认会按 key 排序，这里保证了 trends 的 index 是有序的
    trends = df_calc.groupby(['聚类ID', '年份'])['占比'].mean().unstack()
    
    features = {}
    for cid, row in trends.iterrows():
        ts = row.dropna()
        if len(ts) < 2:
            features[cid] = {'Starting': 'Low', 'Trend': 'Stable', 'Ending': 'Low', 'MeanStart': 0}
            continue
            
        start_val = ts.iloc[:3].mean()
        end_val = ts.iloc[-3].mean()
        slope = end_val - start_val
        std = ts.std()
        
        if start_val > th_high: s_lbl = 'High'
        elif start_val > th_med: s_lbl = 'Medium'
        else: s_lbl = 'Low'
        
        if end_val > th_high: e_lbl = 'High'
        elif end_val > th_med: e_lbl = 'Medium'
        else: e_lbl = 'Low'
        
        if slope > th_slope: t_lbl = 'Rise'
        elif slope < -0.05: t_lbl = 'Decline'
        elif std > (1.0 if is_intensity else 0.1): t_lbl = 'Fluctuate'
        else: t_lbl = 'Stable'
        
        features[cid] = {
            'Starting': s_lbl, 'Trend': t_lbl, 'Ending': e_lbl, 'MeanStart': start_val
        }
    return features

# --- 2. 排序逻辑 (100% 复刻 Cell 3) ---
def sort_clusters_original_logic(sort_df: pd.DataFrame) -> pd.DataFrame:
    """完全使用 Cell 3 的排序 Key 逻辑"""
    orders = {
        'Starting': {'Low': 0, 'Medium': 1, 'High': 2},
        'Ending': {'Low': 0, 'Medium': 1, 'High': 2},
        'Trend': {'Rise': 0, 'Stable': 1, 'Fluctuate': 2, 'Decline': 3}
    }
    
    df = sort_df.copy()
    df['sort_key'] = df.apply(lambda r: (
        orders['Starting'].get(r['Starting'], 99),
        orders['Ending'].get(r['Ending'], 99),
        orders['Trend'].get(r['Trend'], 99),
        r.get('MeanStart', 0)
    ), axis=1)
    
    return df.sort_values('sort_key')

# --- 3. 主执行流程 ---
def process_and_lock_strict():
    print("🚀 开始执行：生成数据 -> 全量排序 -> 锁定 ID")
    
    all_records = []
    
    # === 步骤 A: 生成所有数据 (模拟 Cell 12 的写入过程) ===
    tasks = [
        ("3-1-L2_Policy_Clustering_Breadth.csv", "Breadth", False),
        ("3-1-L2_Policy_Clustering_Intensity.csv", "Intensity", True)
    ]

    for fname, m_type, is_intensity in tasks:
        fpath = DATA_DIR / fname
        if not fpath.exists():
            print(f"⚠️ 跳过文件: {fname}")
            continue

        df_raw = pd.read_csv(fpath, encoding='utf-8-sig')
        
        # 严格按照 Cell 12 的循环顺序生成记录
        for l2 in df_raw['L2政策中文名'].dropna().unique():
            l2_data = df_raw[df_raw['L2政策中文名'] == l2]
            feats_map = calculate_cluster_metrics(l2_data, is_intensity)
            
            # 模拟 Cell 12 的 iterrows 展开
            for _, row in l2_data.iterrows():
                cid = row['聚类ID']
                if cid in feats_map:
                    f = feats_map[cid]
                    all_records.append({
                        'L2政策中文名': l2,
                        '国家': row['国家'],
                        '聚类ID': cid,
                        'Starting': f['Starting'],
                        'Trend': f['Trend'],
                        'Ending': f['Ending'],
                        'MeanStart': f['MeanStart'],
                        'Type': m_type
                    })

    if not all_records:
        print("❌ 没有生成任何数据")
        return

    # === 步骤 B: 构建 DataFrame 并去重 (模拟 Cell 12 的输出) ===
    df_full = pd.DataFrame(all_records).drop_duplicates(subset=['L2政策中文名', '国家', 'Type'])
    
    # 保存 CSV (这是 Cell 3 读取的对象)
    df_full.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(f"✅ CSV 已保存: {OUTPUT_CSV.name}")

    # === 步骤 C: 模拟 Cell 3 的读取和排序过程 ===
    # 为了绝对严谨，我们直接对 df_full 进行操作，就像 Cell 3 读进来一样
    order_map = {}
    
    # 1. 筛选 metric (Cell 3 是分开跑的，这里我们循环跑)
    for m_type in ['Breadth', 'Intensity']:
        df_metric = df_full[df_full['Type'] == m_type].copy()
        if df_metric.empty: continue
        
        # 2. 全量排序 (Cell 3 的关键步骤: sort_clusters(df_meta))
        # 这会根据 Starting/Ending 等对整个表排序
        df_metric_sorted = sort_clusters_original_logic(df_metric)
        
        # 3. 循环 L2 提取 ID (Cell 3 的绘图循环)
        for l2 in df_metric_sorted['L2政策中文名'].unique():
            # 提取该 L2 下的所有行
            l2_sort_info = df_metric_sorted[df_metric_sorted['L2政策中文名'] == l2]
            
            # Cell 3 里的逻辑: l2_sort_info.sort_values('sort_key')['聚类ID'].unique()
            # 虽然 df_metric_sorted 已经是排好序的，但 Cell 3 里为了保险又 sort 了一次
            # 我们这里也照做，确保行为一致
            final_sorted_df = sort_clusters_original_logic(l2_sort_info)
            sorted_ids = final_sorted_df['聚类ID'].unique().tolist()
            
            order_map[f"{m_type}_{l2}"] = sorted_ids

    # === 步骤 D: 锁定 JSON ===
    with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
        json.dump(order_map, f, ensure_ascii=False, indent=2)
    print(f"✅ 顺序已严格锁定至 JSON: {OUTPUT_JSON.name}")

# 运行
process_and_lock_strict()

🚀 开始执行：生成数据 -> 全量排序 -> 锁定 ID
✅ CSV 已保存: 3-2-Automated_Recognition_Mode.csv
✅ 顺序已严格锁定至 JSON: fixed_ppt_order.json


In [28]:
# ==========================================
# Cell 2: 读取 JSON 顺序绘图
# ==========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator, MultipleLocator
from pathlib import Path
import json
import matplotlib as mpl

# --- 基础配置 ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
ORDER_FILE = DATA_DIR / "fixed_ppt_order.json"
FEATURE_FILE = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
OUTPUT_DIR = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic"

# --- 样式设置 ---
def setup_mpl_single2():
    mpl.rc('font', size=25)
    mpl.rcParams.update({
        'legend.fontsize': 'small', 'xtick.labelsize': 'small', 'ytick.labelsize': 'small',
        'lines.linewidth': 2, 'axes.linewidth': 2, 'xtick.major.pad': '12', 'ytick.major.pad': '12',
        'xtick.direction': 'in', 'ytick.direction': 'in', 'xtick.top': False, 'ytick.right': False,
        'mathtext.default': 'regular', 'axes.titlesize': 'small'
    })
setup_mpl_single2()

NATURE_COLORS = [
    '#E64B35', "#6917C2", '#00A087', '#3C5488', '#F39B7F', 
    '#8491B4', '#91D1C2', '#DC0000', '#7E6148', '#B09C85', 
    '#E18727', '#20854E', '#0072B5', '#BC3C29', '#6F99AD'
]

def lighten_color(color, amount=0.7):
    try:
        c = mcolors.to_rgb(color)
        return tuple([c[i] + (1 - c[i]) * amount for i in range(3)])
    except: return (0.5, 0.5, 0.5)

# --- 绘图核心 ---
def plot_fixed_order(l2_name, df_data, df_feats, sorted_ids, metric_label):
    # 数据准备
    df_plot = df_data[(df_data['年份'] >= 2005) & (df_data['年份'] <= 2023)]
    years = sorted(df_plot['年份'].unique())
    matrix = df_plot.pivot(index='年份', columns='国家', values='占比').reindex(years)
    overall_mean = matrix.mean(axis=1)
    y_max = matrix.max().max() * 1.15

    # 画布设置
    n_clusters = len(sorted_ids)
    n_cols, n_rows = 3, (n_clusters + 2) // 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 3.5 * n_rows), squeeze=False)
    
    # --- 严格按照 JSON 给定的 ID 顺序循环 ---
    for idx, cid in enumerate(sorted_ids):
        ax = axes[idx // n_cols, idx % n_cols]
        
        # 找对应的国家
        countries = [c for c in df_plot[df_plot['聚类ID'] == cid]['国家'].unique() if c in matrix.columns]
        if not countries: continue

        # 找标题 (从特征文件里读)
        feat_row = df_feats[(df_feats['L2政策中文名'] == l2_name) & (df_feats['聚类ID'] == cid)]
        if not feat_row.empty:
            r = feat_row.iloc[0]
            title_str = f"{r['Starting']}+{r['Trend']}+{str(r['Ending']).replace(' Share', '')}"
        else:
            title_str = f"Cluster {cid}"

        # 绘图操作
        color = NATURE_COLORS[idx % len(NATURE_COLORS)] # 颜色只跟顺序有关
        fill_color = lighten_color(color, 0.7)
        sub_matrix = matrix[countries]
        c_mean = sub_matrix.mean(axis=1)

        ax.plot(sub_matrix.index, sub_matrix.values, color=color, alpha=0.25, lw=1.2, zorder=1)
        ax.plot(c_mean.index, c_mean, marker='o', color=color, lw=2.5, ms=7,
                mfc=fill_color, mec=color, mew=1.8, label='Cluster Average',
                markevery=max(1, len(years)//10), zorder=10)
        ax.plot(overall_mean.index, overall_mean, color='#000000', ls='--', lw=2.5,
                label='Overall Average', alpha=0.85, zorder=9, dashes=(3, 2))

        ax.set_title(f"{title_str}\n({len(countries)} countries)", pad=15, ha='center')
        ax.set_ylim(bottom=-(y_max * 0.04), top=y_max)
        ax.xaxis.set_major_locator(MultipleLocator(1))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
        ax.tick_params(axis='x', rotation=90, labelsize=15, pad=5.5)
        
        if idx % n_cols == 0: ax.set_ylabel(metric_label)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.legend(loc='best', frameon=False, fontsize=12)

    # 清理多余子图
    for j in range(n_clusters, n_rows * n_cols): axes[j // n_cols, j % n_cols].axis('off')
    plt.subplots_adjust(wspace=0.25, top=0.85, bottom=0.15, hspace=0.45)
    
    # 保存
    safe_name = l2_name.replace("/", "_").replace(" ", "_").replace(":", "")
    out_dir_m = OUTPUT_DIR / f"Plots_{metric_label}"
    out_dir_m.mkdir(parents=True, exist_ok=True)
    out_path = out_dir_m / f"{safe_name}_{metric_label}_Sorted.png"
    plt.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    print(f"  -> Saved: {out_path.name}")

# --- 主执行 ---
def main_plot(fname, m_label):
    print(f"\n>>> 开始绘图: {m_label}")
    if not ORDER_FILE.exists() or not FEATURE_FILE.exists():
        print("❌ 请先运行 Cell 1")
        return
        
    with open(ORDER_FILE, 'r', encoding='utf-8') as f:
        order_map = json.load(f)
    df_feats = pd.read_csv(FEATURE_FILE, encoding='utf-8-sig')
    
    fpath = DATA_DIR / fname
    if not fpath.exists(): return
    df_data = pd.read_csv(fpath, encoding='utf-8-sig')
    
    for l2 in df_data['L2政策中文名'].dropna().unique():
        key = f"{m_label}_{l2}"
        if key in order_map:
            plot_fixed_order(l2, df_data[df_data['L2政策中文名']==l2], df_feats, order_map[key], m_label)

# 运行
main_plot("3-1-L2_Policy_Clustering_Breadth.csv", "Breadth")
main_plot("3-1-L2_Policy_Clustering_Intensity.csv", "Intensity")


>>> 开始绘图: Breadth
  -> Saved: Buildings_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Buildings_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: Climate_governance_Breadth_Sorted.png
  -> Saved: Electricity_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Electricity_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: Fossil_fuel_production_policies_Breadth_Sorted.png
  -> Saved: GHG_emissions_data_and_reporting_Breadth_Sorted.png
  -> Saved: GHG_emissions_targets_Breadth_Sorted.png
  -> Saved: Industry_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Industry_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: International_climate_co-operation_Breadth_Sorted.png
  -> Saved: International_public_finance_Breadth_Sorted.png
  -> Saved: Public_Research,_Development_and_Demonstration_Breadth_Sorted.png
  -> Saved: Transport_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Transport_–_non_market-based_instruments_Brea

In [29]:
# ==========================================
# Cell 3: 生成 PPT (Strict Order Matching)
# ==========================================
import pandas as pd
import json
from pathlib import Path
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN

# --- 路径配置 (需与 Cell 1 & 2 保持一致) ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
ORDER_FILE = DATA_DIR / "fixed_ppt_order.json"
FEATURE_FILE = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
IMG_BASE_DIR = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic"
OUTPUT_PPT_DIR = DATA_DIR / "3-3-PPT_Reports"
OUTPUT_PPT_DIR.mkdir(parents=True, exist_ok=True)

# --- 辅助函数：创建表格 ---
def create_table(slide, df_policy, sorted_ids, l2_name):
    """
    在幻灯片底部创建一个表格，严格按照 sorted_ids 的顺序显示特征
    """
    # 筛选并按照 locked order 排序数据
    rows = []
    for cid in sorted_ids:
        # 找到对应 Cluster 的元数据
        row_data = df_policy[df_policy['聚类ID'] == cid]
        if row_data.empty: continue
        
        # 取第一行（因为元数据里有多个国家，特征是一样的）
        r = row_data.iloc[0]
        
        # 统计该聚类下的国家数量
        countries = df_policy[df_policy['聚类ID'] == cid]['国家'].unique()
        country_count = len(countries)
        country_str = ", ".join(countries)
        if len(country_str) > 50: country_str = country_str[:47] + "..." # 截断过长文本
        
        rows.append([
            f"Cluster {cid}",
            f"{r['Starting']} -> {r['Trend']} -> {r['Ending']}",
            f"{r['MeanStart']:.3f}",
            f"{country_count} ({country_str})"
        ])

    # 如果没有数据，直接返回
    if not rows: return

    # 表格位置与尺寸
    rows_count = len(rows) + 1 # +1 for header
    cols_count = 4
    left = Inches(0.5)
    top = Inches(4.5) # 图片通常在上方，表格放下面
    width = Inches(9.0)
    height = Inches(0.8)

    table = slide.shapes.add_table(rows_count, cols_count, left, top, width, height).table

    # 设置列宽
    table.columns[0].width = Inches(1.2) # Cluster ID
    table.columns[1].width = Inches(2.5) # Features
    table.columns[2].width = Inches(1.2) # Mean Start
    table.columns[3].width = Inches(4.1) # Countries

    # 表头
    headers = ["ID", "Trajectory Features", "Mean Start", "Countries"]
    for i, h in enumerate(headers):
        cell = table.cell(0, i)
        cell.text = h
        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(220, 220, 220)
        cell.text_frame.paragraphs[0].font.bold = True
        cell.text_frame.paragraphs[0].font.size = Pt(10)

    # 填充内容
    for r_idx, row_content in enumerate(rows):
        for c_idx, text in enumerate(row_content):
            cell = table.cell(r_idx + 1, c_idx)
            cell.text = str(text)
            cell.text_frame.paragraphs[0].font.size = Pt(9)
            cell.vertical_anchor = 3 # Middle

# --- 主 PPT 生成逻辑 ---
def generate_ppt_for_metric(metric_label: str):
    print(f"\n>>> 正在生成 PPT: {metric_label}")
    
    # 1. 检查文件
    if not ORDER_FILE.exists() or not FEATURE_FILE.exists():
        print("❌ 缺少 JSON 或 CSV 文件，请先运行 Cell 1")
        return

    # 2. 读取数据
    with open(ORDER_FILE, 'r', encoding='utf-8') as f:
        order_map = json.load(f)
    
    df_all_feats = pd.read_csv(FEATURE_FILE, encoding='utf-8-sig')
    df_feats = df_all_feats[df_all_feats['Type'] == metric_label]

    # 3. 初始化 PPT
    prs = Presentation()
    
    # 设置宽屏 (16:9)
    prs.slide_width = Inches(13.333)
    prs.slide_height = Inches(7.5)

    # 4. 遍历所有在该 metric 下的政策
    #    我们直接从 DataFrame 中获取政策列表，确保不漏
    policy_list = df_feats['L2政策中文名'].unique()
    
    count = 0
    for l2 in policy_list:
        key = f"{metric_label}_{l2}"
        
        # 如果这个政策不在我们的锁定列表中（可能是数据太少被过滤了），跳过
        if key not in order_map:
            continue
            
        sorted_ids = order_map[key] # 获取锁定的 ID 顺序
        
        # --- 新建幻灯片 ---
        slide = prs.slides.add_slide(prs.slide_layouts[6]) # 空白布局
        
        # --- 标题 ---
        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(12), Inches(0.5))
        tf = title_box.text_frame
        tf.text = f"{l2} ({metric_label})"
        tf.paragraphs[0].font.bold = True
        tf.paragraphs[0].font.size = Pt(24)
        
        # --- 插入图片 ---
        # 图片路径必须和 Cell 2 生成的完全一致
        safe_name = l2.replace("/", "_").replace(" ", "_").replace(":", "")
        img_path = IMG_BASE_DIR / f"Plots_{metric_label}" / f"{safe_name}_{metric_label}_Sorted.png"
        
        if img_path.exists():
            # 放置图片 (居中靠上)
            # 原图很大，我们缩小一点放上面
            slide.shapes.add_picture(str(img_path), Inches(0.5), Inches(0.8), width=Inches(12.3))
        else:
            # 如果没找到图，放个提示
            err_box = slide.shapes.add_textbox(Inches(4), Inches(3), Inches(5), Inches(1))
            err_box.text_frame.text = "Image Not Found (Check Cell 2 Output)"

        # --- 插入数据表格 (关键：按锁定顺序) ---
        # 筛选当前政策的数据
        df_policy = df_feats[df_feats['L2政策中文名'] == l2]
        create_table(slide, df_policy, sorted_ids, l2)
        
        count += 1

    # 5. 保存
    output_path = OUTPUT_PPT_DIR / f"Report_{metric_label}_Sorted.pptx"
    prs.save(output_path)
    print(f"✅ PPT 已保存 ({count} 页): {output_path.name}")

# --- 执行 ---
generate_ppt_for_metric("Breadth")
generate_ppt_for_metric("Intensity")


>>> 正在生成 PPT: Breadth
✅ PPT 已保存 (15 页): Report_Breadth_Sorted.pptx

>>> 正在生成 PPT: Intensity
✅ PPT 已保存 (15 页): Report_Intensity_Sorted.pptx


In [26]:
import pandas as pd
from pptx import Presentation
from pathlib import Path
from pptx.util import Inches
from typing import List, Optional, Dict
import re
import json

# ==========================================
# 1. 强力清洗函数 (保留原逻辑)
# ==========================================
def normalize_title_robust(text: str) -> str:
    """
    终极清洗：转小写、替换所有类型横杠、去多余空格
    """
    if not isinstance(text, str): return str(text) if text is not None else ""
    text = text.replace('\u2013', '-').replace('\u2014', '-').replace('–', '-')
    text = text.replace('\xa0', ' ').replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

def normalize_spaces(text: str) -> str:
    """清理文本框内容中的特殊空格"""
    return re.sub(r'\s+', ' ', text).strip()

def get_slide_title(slide) -> Optional[str]:
    """获取幻灯片标题"""
    if slide.shapes.title and slide.shapes.title.text.strip():
        return slide.shapes.title.text.strip()
    
    candidates = []
    for shape in slide.shapes:
        if shape.has_text_frame and shape.text_frame.text.strip():
            candidates.append((shape.top, shape.text_frame.text.strip()))
    
    if candidates:
        candidates.sort(key=lambda x: x[0])
        if candidates[0][0] < Inches(2.0):
            return candidates[0][1]
    return None

# ==========================================
# 2. 核心：加载 JSON 锁定顺序
# ==========================================
def load_fixed_order(json_path: Path) -> Dict[str, List[int]]:
    if not json_path.exists():
        print(f"❌ JSON 顺序文件未找到: {json_path}")
        return {}
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# ==========================================
# 3. 核心同步逻辑 (利用 JSON 回填)
# ==========================================
def sync_ppt_with_json(ppt_path: Path, csv_path: Path, json_path: Path, target_type: str) -> None:
    print(f"\n{'='*50}")
    print(f"📂 处理文件: {ppt_path.name}")
    print(f"🎯 目标类型: {target_type}")
    print(f"🔐 顺序参照: {json_path.name}")
    print(f"{'='*50}")
    
    if not ppt_path.exists():
        print(f"❌ PPT文件不存在: {ppt_path}")
        return

    prs = Presentation(ppt_path)
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    order_map = load_fixed_order(json_path)
    
    if not order_map:
        print("❌ 无法加载顺序映射，终止操作。")
        return
    
    # === 关键1：创建匹配列 ===
    df['Match_Title'] = df['L2政策中文名'].apply(normalize_title_robust)
    
    # 同时也把 JSON 的 key 处理一下，方便查找
    # JSON key 格式: "Breadth_PolicyName" -> 我们存一个映射: clean_policy_name -> [ids]
    json_lookup = {}
    prefix = f"{target_type}_"
    for key, ids in order_map.items():
        if key.startswith(prefix):
            raw_policy_name = key[len(prefix):] # 去掉 "Breadth_" 前缀
            clean_name = normalize_title_robust(raw_policy_name)
            json_lookup[clean_name] = ids

    update_count = 0
    
    for slide_idx, slide in enumerate(prs.slides):
        raw_ppt_title = get_slide_title(slide)
        if not raw_ppt_title: continue
        
        # 处理 PPT 标题（有些标题可能包含括号里的 metric type，去掉它）
        title_pure = raw_ppt_title.replace(f"({target_type})", "").strip()
        clean_ppt_title = normalize_title_robust(title_pure)
        
        # 1. 查找 JSON 里的顺序
        if clean_ppt_title not in json_lookup:
            # 尝试模糊匹配 (比如 PPT 标题不完整)
            found = False
            for k in json_lookup.keys():
                if k in clean_ppt_title or clean_ppt_title in k:
                    clean_ppt_title = k
                    found = True
                    break
            if not found:
                # print(f"⚠️ [无顺序记录] PPT标题: '{raw_ppt_title}' - 跳过")
                continue
        
        sorted_cids = json_lookup[clean_ppt_title]
        
        # 2. 获取 PPT 里的标签框 (从左到右排序)
        # 筛选逻辑：有文本，且文本里包含 '+' 号 (特征标识)
        label_shapes = [s for s in slide.shapes if s.has_text_frame and "+" in s.text_frame.text]
        # 排序：先按 Top (行)，再按 Left (列)。允许多行布局。
        # 容差：Top 差异在 0.5 英寸内视为同一行
        label_shapes.sort(key=lambda s: (int(s.top / Inches(0.5)), s.left))
        
        if len(label_shapes) != len(sorted_cids):
            # 这种情况比较少见，可能是 PPT 被人手动删改了
            print(f"⚠️ [数量不符] {title_pure}: PPT有{len(label_shapes)}个框, JSON记录有{len(sorted_cids)}个聚类")
            continue
            
        # 3. 逐一回填
        for shape, cid in zip(label_shapes, sorted_cids):
            raw_text = normalize_spaces(shape.text_frame.text)
            
            # 解析 PPT 文本内容: "High + Stable + Low (xx countries)"
            parts = raw_text.split('+')
            if len(parts) >= 3:
                p1 = parts[0].strip()
                p2 = parts[1].strip()
                p3_full = parts[2].strip()
                
                # 处理 Ending 部分，去掉括号里的国家数
                final_e = p3_full
                last_paren = p3_full.rfind('(')
                if last_paren != -1:
                    final_e = p3_full[:last_paren].strip()
                
                # 清洗 "Share" 后缀
                new_s = p1.replace(' Share', '')
                new_t = p2
                new_e = final_e.replace(' Share', '')
                
                # 4. 更新 CSV
                # 定位条件：标准化的政策名 + 类型 + 聚类ID (这个ID来自 JSON，绝对准确)
                target_idx = df[(df['Match_Title'] == clean_ppt_title) & 
                                (df['Type'] == target_type) & 
                                (df['聚类ID'] == cid)].index
                
                if not target_idx.empty:
                    # 检查是否需要更新
                    old_vals = df.loc[target_idx[0], ['Starting', 'Trend', 'Ending']].values
                    new_vals = [new_s, new_t, new_e]
                    
                    if list(old_vals) != new_vals:
                        print(f" 🔄 [更新] {title_pure[:15]}... (ID:{cid})")
                        print(f"    旧: {old_vals} -> 新: {new_vals}")
                        
                        df.loc[target_idx, 'Starting'] = new_s
                        df.loc[target_idx, 'Trend'] = new_t
                        df.loc[target_idx, 'Ending'] = new_e
                        update_count += len(target_idx) # 可能一次更新多行(多个国家)

    # 清理
    if 'Match_Title' in df.columns: del df['Match_Title']
    
    # 保存
    if update_count > 0:
        try:
            df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print(f"✅ 保存成功！共更新了 {update_count} 行数据。")
        except PermissionError:
            print("❌ 保存失败！请先关闭 CSV 文件！")
    else:
        print("✨ 文件已是最新，无需更新。")

# ==========================================
# 4. 主程序
# ==========================================
def main():
    base_dir = Path.cwd().parent
    data_dir = base_dir / "data" if (base_dir / "data").exists() else Path.cwd() / "data"
    
    # 路径配置
    # 注意：这里假设你的人工修改后的 PPT 放在这个 Human_catorgy 文件夹下
    ppt_dir = data_dir / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "Human_catorgy"
    csv_file = data_dir / "3-2-Automated_Recognition_Mode.csv"
    json_file = data_dir / "fixed_ppt_order.json"
    
    # 1. 更新 Breadth
    sync_ppt_with_json(ppt_dir / "Analysis_Report_Breadth_Editable.pptx", csv_file, json_file, "Breadth")
    
    # 2. 更新 Intensity
    sync_ppt_with_json(ppt_dir / "Analysis_Report_Intensity_Editable.pptx", csv_file, json_file, "Intensity")

if __name__ == "__main__":
    main()


📂 处理文件: Analysis_Report_Breadth_Editable.pptx
🎯 目标类型: Breadth
🔐 顺序参照: fixed_ppt_order.json
✨ 文件已是最新，无需更新。

📂 处理文件: Analysis_Report_Intensity_Editable.pptx
🎯 目标类型: Intensity
🔐 顺序参照: fixed_ppt_order.json
✨ 文件已是最新，无需更新。
